In [ ]:
import pandas as pd
import numpy as np
import itertools
import json
import pickle

In [2]:
from tqdm import tqdm
tqdm.pandas()

In [3]:
folder="dataset_v5/v5_2/"

In [4]:
ment = pd.read_csv(folder+"processed_sodhganga_mentorship_dept_rev_with_initial_ids.csv")

In [5]:
#ment.loc[ment["uniq_ddc_across_dept"]=="('',)",'DepartmentId']=ment[ment["uniq_ddc_across_dept"]=="('',)"]['N_DepartmentId_1']

In [6]:
#ment[ment]

In [7]:
ment.shape

(205744, 25)

In [8]:
ment.columns

Index(['advisorId', 'researcherId', 'advisor_name', 'researcher_name',
       'dc.language.iso[]', 'publisher_dept', 'DepartmentId',
       'publisher_institution', 'instituteId', 'title', 'thesisId',
       'dc.type.degree', 'date_submitted', 'dc.date.awarded', 'dc.subject.ddc',
       'dc.description.abstract', 'dc.title.alternative',
       'uniq_ddc_across_dept', 'N_DepartmentId_1', 'N_DepartmentId',
       'previous_advisor_name', 'advisor_inst_dept', 'res_inst_dept',
       'adv_names_rid', 'res_names_rid'],
      dtype='object')

In [9]:
chars=[',',";","\.","-",":"]

In [10]:
def save_obj(obj, name ):
    with open(folder+name+'.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [11]:
def rm_splChar(name):
    name = str(name)
    name1 = re.sub(" +","",name)
    regex = "|".join(chars)
    name1 = re.sub(regex,"", name1)
    #name2 = re.sub(regex,"", name)
    #val = re.sub('[^A-Za-z]+', '', val)
    return name1

In [12]:
def check_and_combine1(name_dict,ment):
    for key in name_dict:
        if len(name_dict[key])==1:
            global pairs_corrected
            #print(key,name_dict[key])
            global merged_count
            pairs_list=name_dict[key][0]
            #pairs_corrected.append(piars_tuple)
            merged_count+=1
            id1=name_dict[key][0][1]
            id2=name_dict[key][0][3]
            #print(id1,id2)
            as_stud1=ment[ment['researcherId']==id1]
            as_stud2=ment[ment['researcherId']==id2]
            if as_stud1.empty and as_stud2.empty:
                pairs_corrected.append(pairs_list)
            elif not (as_stud1.empty) and as_stud2.empty:
                pairs_corrected.append(pairs_list)
            elif as_stud1.empty and not(as_stud2.empty):
                #ment.loc[(ment['resId']==id2),'resId']=id1
                pairs_corrected.append(pairs_list)
            else:
                print("check ids: "+str(id1)+"_"+str(id2))
        else:
            continue
    save_obj(pairs_corrected,'sim_name_in_operator')
    return

In [13]:
def check_and_combine(name_dict,ment):
    global pairs_corrected
    global merged_count
    for key in name_dict:
        if len(name_dict[key])==1:
            #print(key,name_dict[key])
            pairs_list=[name_dict[key][0][0],name_dict[key][0][1],name_dict[key][0][2],name_dict[key][0][3]]
            #pairs_corrected.append(piars_tuple)
            merged_count+=1
            id1=name_dict[key][0][0]
            id2=name_dict[key][0][2]
            #print(id1,id2)
            as_stud1=ment[ment['researcherId']==id1]
            as_stud2=ment[ment['researcherId']==id2]
            if as_stud1.empty and as_stud2.empty:
                ment.loc[(ment['advisorId']==id2),'advisorId']=id1
                pairs_corrected.append(pairs_list)
            elif not (as_stud1.empty) and as_stud2.empty:
                ment.loc[(ment['advisorId']==id1),'advisorId']=id2
                ment.loc[(ment['researcherId']==id1),'researcherId']=id2
                pairs_corrected.append(pairs_list)
            elif as_stud1.empty and not(as_stud2.empty):
                ment.loc[(ment['advisorId']==id1),'advisorId']=id2
                #ment.loc[(ment['resId']==id2),'resId']=id1
                pairs_corrected.append(pairs_list)
            else:
#                 ment.loc[(ment['advId']==id1),'advId']=id2
#                 ment.loc[(ment['resId']==id1),'resId']=id2
                print("check ids: "+str(id1)+"_"+str(id2))
        else:
            continue
    pairs_df=pd.DataFrame(pairs_corrected)
    pairs_df.to_csv(folder+"/corrected_names_using_in_operator1.csv",sep=",",index=False)
    return

In [14]:
# def write_dict(data,inst_id,dept_id):
#     #print(data)
#     file_name=str(inst_id)+"_"+str(dept_id)
#     with open(folder+'adv_dict/'+file_name+'.json', 'w') as fp:
#         json.dumps({str(k): v for k, v in data.items()})
#     return

In [15]:
# converted={ast.literal_eval(k): v for k, v in json.items()}

In [16]:
def join_advisors(name_id,ment):
    global mem_dict
    #print(name_id)
    dept_id=pd.unique(name_id[0])[0]
    inst_id=pd.unique(name_id[1])[0]
    tmp_df=pd.DataFrame({'name':name_id[2],'idd':name_id[3]})
    idd_grp=tmp_df.groupby(['idd'],as_index=False)[['name']].agg(lambda x:list(x))
    idd_grp['name']=idd_grp['name'].apply(lambda x: x[0])
    #print(idd_grp.head())
    
    name_comb = itertools.combinations(idd_grp['name'], 2)
    id_comb = itertools.combinations(idd_grp['idd'], 2)
    for name, idd in zip(name_comb, id_comb):
        name0=name[0]
        name1=name[1]
        idd0=idd[0]
        idd1=idd[1]
        if len(name0) > len(name1):
            name0,name1= name1,name0
            idd0,idd1=idd1,idd0
        if (name0 in name1) and (idd0 != idd1):# or (name[1] in name[0]))
            if idd0 in mem_dict:
                mem_dict[idd0].append((name0,idd0,name1,idd1))
            else:
                mem_dict[idd0]=[]
                mem_dict[idd0].append((name0,idd0,name1,idd1))
#     if len(mem_dict)>0:
#         write_dict(mem_dict,inst_id,dept_id)
#         check_and_combine(mem_dict,ment)
#     print(mem_dict)
    return

In [ ]:
# def join_advisors1(name_id,ment):
#     dept_id=pd.unique(name_id[0])[0]
#     inst_id=pd.unique(name_id[1])[0]
#     tmp_df=pd.DataFrame({'name':name_id[2],'idd':name_id[3]})
#     idd_grp=tmp_df.groupby(['idd'],as_index=False)[['name']].agg(lambda x:list(x))
#     idd_grp['name']=idd_grp['name'].apply(lambda x: set(x))
#     id_comb = itertools.combinations(idd_grp['idd'], 2)
#     for id1, id2 in id_comb:
#         name1_list=idd_grp[idd_grp['idd']==id1]['name'].values[0]
#         name2_list=idd_grp[idd_grp['idd']==id2]['name'].values[0]
#         inner_function(name1_list,name2_list,id1,id2)
#     return

In [ ]:
# def inner_function(name1_list,name2_list,idd0,idd1):
#     global mem_dict
#     product=itertools.product(name1_list, name2_list)
#     #print(list(product))
#     for name0, name1 in product:
#         if len(name0) > len(name1):
#             name0,name1= name1,name0
#             idd0,idd1=idd1,idd0
#         if (name0 in name1) and (idd0 != idd1):
#             #print(name0+"__"+name1)
#             if idd0 in mem_dict:
#                 mem_dict[idd0].append((name0,idd0,name1,idd1))
#             else:
#                 mem_dict[idd0]=[]
#                 mem_dict[idd0].append((name0,idd0,name1,idd1)
#             return
#         else:
#             continue
#     return

In [17]:
def join_advisors1(name_id,ment):
    dept_id=pd.unique(name_id[1])[0]
    inst_id=pd.unique(name_id[0])[0]
    tmp_df=pd.DataFrame({'name':name_id[2],'idd':name_id[3]}).copy()
    idd_grp=tmp_df.groupby(['idd'],as_index=False)[['name']].agg(lambda x:list(x))
    idd_grp['name']=idd_grp['name'].apply(lambda x: set(x))
    id_comb = itertools.combinations(idd_grp['idd'], 2)
    for id1, id2 in id_comb:
        name1_list=idd_grp[idd_grp['idd']==id1]['name'].values[0]
        name2_list=idd_grp[idd_grp['idd']==id2]['name'].values[0]
        product=itertools.product(name1_list, name2_list)
        #print(list(product))
        for name1, name2 in product:
            if len(name1) > len(name2):
                name1,name2 = name2,name1
                id11,id21 = id2,id1
            else:
                id11=id1
                id21=id2
            if name1 in name2:
#                 print(name1+"__"+name2)
#                 print(str(id1)+"__"+str(id2))
                if id1 in mem_dict:
                    mem_dict[id11].append((name1,id11,name2,id21))
                else:
                    mem_dict[id11]=[]
                    mem_dict[id11].append((name1,id11,name2,id21))
                break
            else:
                continue
#         inner_function(name1_list,name2_list,id1,id2)
    return

In [18]:
advisors=ment.groupby(['instituteId','N_DepartmentId_1'],as_index=False)[['advisor_name','advisorId']].agg(lambda x: list(x))

In [19]:
advisors.shape

(6938, 4)

In [1]:
# mem_dict={}
# tmp=advisors[['instituteId','N_DepartmentId_1','advisor_name','advisorId']].progress_apply(join_advisors1,args=(ment,),axis=1)

In [ ]:
pairs_corrected=[]
check_and_combine1(mem_dict, ment)

In [ ]:
count=0
for key in mem_dict:
    count+=1
    if key==46623:
        print(mem_dict[key])
        break

In [ ]:
list(mem_dict.values())[999:]

In [ ]:
# 85852: [('Srivastava', 85852, 'Srivastava, S. K.', 85837),
#   ('Srivastava', 85852, 'Srivastava, Santosh K.', 85839),
#   ('Srivastava', 85852, 'Srivastava, Santosh Kumar', 85855),
#   ('Srivastava', 85852, 'Srivastava, Santhosh Kumar', 85891),
#   ('Srivastava', 85852, 'Srivastava, S. D.', 85892),
#   ('Srivastava', 85852, 'Srivastava, S. P.', 85893),
#   ('Srivastava', 85852, 'Srivastava, O. P.', 85914),
#   ('Srivastava', 85852, 'Srivastava, H. N.', 85917)],

In [ ]:
len(mem_dict)

In [ ]:
#{(201704, 'Narayana, N.'): [(30762, 'Narayana, N. V.')]}

In [ ]:
ment[(ment['instituteId']=='I160') & (ment['DepartmentId']=='D1331') & (ment['advisor_name']=='Yadava')].values

In [ ]:
ment[ment['advisorId']==39544]

In [ ]:
ment[ment['advisorId']==23258]#['title'].values

In [ ]:
#  [34772, 'Mahindroo', 34775, 'Neeraj, Mahindroo'],
#  [34773, 'Neeraj', 34775, 'Neeraj, Mahindroo'],

In [ ]:
len(pairs_corrected)

In [ ]:
pairs_corrected[999:]

In [ ]:
count=0
for a in pairs_corrected:
    count+=1
    if a[0]==55747:
        print(count)
        print(a)

In [ ]:
pairs_sorted=sorted(pairs_corrected,key=lambda x: x[0])

In [ ]:
for a in pairs_corrected:
    if a[0]==197209:
        print(a)

In [ ]:
#mem_dict

In [ ]:
# import collections
# print([item for item, count in collections.Counter(first).items() if count > 1])

In [ ]:
ment[ment['researcherId']==23421][['title','date_submitted']]

In [ ]:
ment[ment['researcherId']==23258][['title','date_submitted']]

In [ ]:
ment.columns

In [ ]:
ment[ment['advisorId']==85911]['title'].values[0]